Machine learning tasks are typically data heavy, requiring either labelled data for supervised learning or unlabelled data for unsupervised learning. In this user guide, the [`intake`](https://github.com/ContinuumIO/intake) library is used to fetch large datasets from remote data sources efficiently, including built in caching to avoid unncessary download when the files are available locally.

Once you have loaded your data, you will typically need to reshape it appropriately before it can be fed into a machine learning pipeline. These steps are detailed in the next user guide [Alignment_and_Preprocessing](03_Alignment_and_Preprocessing.ipynb).